In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from ASHRAE_tools import meter_dict
import ASHRAE_data_prep

meter_dict = meter_dict()

## Modelo de Regresión 1

In [6]:
X_train, y_train = ASHRAE_data_prep.train_reg()

In [4]:
X_train.dtypes

building_id             int64
meter                   int64
site_id                 int64
square_feet             int64
air_temperature       float64
dew_temperature       float64
sea_level_pressure    float64
wind_direction        float64
wind_speed            float64
hour                    int64
wday                    int64
week                    int64
dtype: object

In [4]:
X_train.describe()

,building_id,meter,site_id,square_feet,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed,hour,wday,week
count,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07
mean,7.992780e+02,6.624412e-01,7.992232e+00,1.077830e+05,1.595523e+01,7.729669e+00,1.016080e+03,1.713399e+02,3.377227e+00,1.150232e+01,3.006958e+00,2.692979e+01
std,4.269133e+02,9.309921e-01,5.099060e+00,1.171424e+05,1.096630e+01,1.018906e+01,6.925514e+00,1.128653e+02,2.263302e+00,6.922017e+00,1.997191e+00,1.503481e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,2.830000e+02,-2.890000e+01,-3.500000e+01,9.682000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,3.930000e+02,0.000000e+00,3.000000e+00,3.252700e+04,8.300000e+00,0.000000e+00,1.011800e+03,7.285714e+01,2.100000e+00,6.000000e+00,1.000000e+00,1.400000e+01
50%,8.950000e+02,0.000000e+00,9.000000e+00,7.270900e+04,1.670000e+01,8.900000e+00,1.016100e+03,1.700000e+02,3.100000e+00,1.200000e+01,3.000000e+00,2.700000e+01
75%,1.179000e+03,1.000000e+00,1.300000e+01,1.391130e+05,2.390000e+01,1.610000e+01,1.020300e+03,2.700000e+02,4.600000e+00,1.800000e+01,5.000000e+00,4.000000e+01
max,1.448000e+03,3.000000e+00,1.500000e+01,8.750000e+05,4.720000e+01,2.610000e+01,1.045500e+03,3.600000e+02,1.900000e+01,2.300000e+01,6.000000e+00,5.300000e+01


In [5]:
X_train.head()

,building_id,meter,site_id,square_feet,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed,hour,wday,week
0,0,0,0,7432,25.0,20.0,1019.7,0.0,0.0,0,4,53
1,0,0,0,7432,24.4,21.1,1020.2,70.0,1.5,1,4,53
2,0,0,0,7432,22.8,21.1,1020.2,0.0,0.0,2,4,53
3,0,0,0,7432,21.1,20.6,1020.1,0.0,0.0,3,4,53
4,0,0,0,7432,20.0,20.0,1020.0,250.0,2.6,4,4,53


In [7]:
mod_reg = LinearRegression()

In [8]:
mod_reg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [10]:
del X_train

In [8]:
mod_reg.coef_

array([[-2.65885923e+01,  2.08145493e+03,  2.34789920e+03,
         2.92220651e-02, -1.17494723e+02,  1.39814835e+02,
         2.28839207e+00, -3.57551059e+00,  5.70947189e+02,
         1.05215779e+01, -5.52471680e+01, -7.39483025e+01]])

In [3]:
row, X_test = ASHRAE_data_prep.test_reg()

In [4]:
X_test.shape

(41697600, 12)

In [9]:
y = mod_reg.predict(X_test)

In [11]:
y.shape

(41697600, 1)

In [12]:
row.shape

(41697600, 1)

AttributeError: 'DataFrame' object has no attribute 'type'

In [34]:
row['meter_reading'] = np.maximum(0,y)

In [25]:
row.sort_values('row_id', inplace = True)

In [26]:
row.head()

,row_id,meter_reading
0,0,-1114.938429
17520,1,-1279.221392
35040,2,-1228.196179
52560,3,-719.757981
70080,4,1969.026160


In [29]:
y = np.max(0,y)

TypeError: only integer scalar arrays can be converted to a scalar index

In [35]:
row.head()

,row_id,meter_reading
0,0,0.0
17520,1,0.0
35040,2,0.0
52560,3,0.0
70080,4,0.0


In [ ]:
row.to_csv('./submission.csv')